In [ ]:
class LogReg:
    
    def __init__(self, train_x, train_y, features, val_x, val_y):
        self.train_x = train_x
        self.train_y = train_y
        self.features = features
        self.val_x = val_x
        self.val_y = val_y
        
    def copy_drop(self, train_x, val_x, drop=None): #drop isn't working - why?
        self.train_x = train_x.drop(columns=self.drop)
        self.val_x = val_x.drop(columns=self.drop)
        self.drop = drop
        return train_x, val_x
        
    def fit_pred(self, train_x, train_y, features, val_x, val_y):
        self.lm = LogisticRegression(C=1e13,penalty='l1',solver='liblinear')
        self.lm_fit = self.lm.fit(train_x[features],train_y)
        self.features = features
        self.preds = self.lm.predict(val_x[features])
        print('Accuracy of logistic regression classifier on val set: {:.2f}'.format(
                                                                            self.lm.score(val_x[features], val_y)))
    
    def get_preds(self):
        return self.preds    
        
    def get_coefs(self):
        print('\n','Feature coefficient results: \n')
        for feature, coef in zip(self.train_x[features].columns, self.lm.coef_.reshape(1, -1)[0]):
            print(feature, ':', f'{coef:.3f}')
            
    def get_proba(self):
        return self.lm.predict_proba(self.val_x[features])
    
    def conf_matrix(self):
        print('\n','Confusion matrix')
        print(confusion_matrix(self.val_y, self.preds),'\n')
        print('Classification report')
        print(classification_report(self.val_y, self.preds))
            
    def roc(self):
        self.val_x['proba_bank'] = self.lm.predict_proba(self.val_x[features])[:, 1]
        self.fpr, self.tpr, self.thresholds = roc_curve(self.val_y, self.val_x['proba_bank'])
        self.auc = roc_auc_score(self.val_y, self.val_x['proba_bank'])
        plt.plot(self.fpr, self.tpr, marker='o')
        plt.xlabel('1 - Specificity (FPR)')
        plt.ylabel('Sensitivity (TPR)');
        plt.title(f"Area Under the ROC Curve: {round(self.auc, 3)}");